In [94]:
from __future__ import division
import pdb

from scipy import optimize
import numpy as np
% matplotlib inline

### Helper Functions

In [91]:
def midpoint_vals(bound):
    """
        Helper method to generate inital value for design variable based on established bounds.
        
        Parameters
        ----------
        bounds : {tuple}
            upper and lower bounds for design variable
    """
    if bound[0] is None:
        return bound[1]
    
    if bound[1] is None:
        return bound[0]
    
    return (bound[1] - bound[0])/2

### Deterministic Model

In [79]:
def deter_obj(x, q, c, e, f):
    """
        Objective function for deterministic problem. Single time horizon - no uncertainty.
        
        Parameters
        ----------
        x : {np.array}
            decision variables
        
        q : {np.array}
            sale price
        
        c : {np.array}
            purchase cost
        
        e : {np.array}
            inventory return
        
        f : {np.array}
            operation cost
    """
    s = x[:2]
    p = x[2:4]
    I = x[4:8]
    R = x[8:]
    
    obj = np.dot(q, s) - np.dot(c,p) + np.dot(e,I) - np.dot(f,R)
    
    return -1*obj

In [89]:
# sales price
q = np.array([100,80])

# purchase cost
c = np.array([10, 15])

# inventory coefficients
e = np.array([10, 15, 20, 25])

# operation cost
f = np.array([30, 35])

# decision variables - s3, s4, p1, p2, I1_f, I2_f, I3_f, I4_f, R1, R2

# bounds
bounds = [(20, 40), (20, 85), (20, 100), (30, 200), (0.0001, 20), (0.0001, 20), (0.0001, 20), (0.0001, 20), (20, 100), (30, 100)]

# init vals - boundary midpoints
x0 = np.array([(tup[1] - tup[0])/2 for tup in bounds])

cons = [
    {"type": "eq", "fun": lambda x: x[2] - 0.4*x[8] - x[4]},
    {"type": "eq", "fun": lambda x: x[3] - 0.6*x[8] - x[9] - x[5]},
    {"type": "eq", "fun": lambda x: 0.5*x[8] - x[6] - x[0]},
    {"type": "eq", "fun": lambda x: 0.6*x[9] - x[7] - x[1]}
]

# generate boundary constraints dynamically
dynamic_cons = []
for idx, bnd in enumerate(bounds):
    dynamic_cons.append({"type": "ineq", "fun": lambda x: x[idx] - bnd[0]})
    dynamic_cons.append({"type": "ineq", "fun": lambda x: bnd[1] - x[idx]})

cons += dynamic_cons

res = optimize.minimize(deter_obj, x0, args=(q,c,e,f), method="SLSQP", bounds=bounds, constraints=cons, options={"disp":True})
    
x = res.x

print "Sales Units - %s" % ", ".join(["S%s: %s" % (idx, val) for idx, val in enumerate(x[:2])])
print "Purchase Units - %s" % ", ".join(["P%s: %s" % (idx, val) for idx, val in enumerate(x[2:4])])
print "Inventory Units - %s" % ", ".join(["I%s_final: %s" % (idx, val) for idx, val in enumerate(x[4:8])])
print "Operation Levels - %s" % ", ".join(["R%s: %s" % (idx, val) for idx, val in enumerate(x[8:])])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -493.320899987
            Iterations: 5
            Function evaluations: 61
            Gradient evaluations: 5
Sales Units - S0: 39.9999999997, S1: 20.0
Purchase Units - P0: 41.0000149999, P1: 88.1667849999
Inventory Units - I0_final: 8.99993500007, I1_final: 6.83316500007, I2_final: 0.000100000090872, I3_final: 0.00010000004724
Operation Levels - R0: 80.0001999996, R1: 33.3335000001


### TSSP - Without Risk

In [172]:
def tssp_obj(x, q, c, e, f, c_susl, num_scenarios):
    """
        Objective function for two stage stochastic problem.
        
        Parameters
        ----------
        x : {np.array}
            design variables
        
        q : {np.array}
            sales price (stochastic) - expected to have shape = (num_scenarios, num_det_var)
        
        c : {np.array}
            purchase costs
        
        e : {np.array}
            inventory returns
        
        f : {np.array}
            operation costs
        
        c_susl : {np.array}
            surplus and slack costs
        
        num_scenarios : {int}
            number of scenarios
    """
    scenario_prob = 1/num_scenarios
    
    # segment design variables
    num_stoch = q.size
    s = x[:num_stoch]
    s = s.reshape(num_scenarios,2)
    
    det_indices = [len(c), len(e), len(f)]
    for idx in range(0, len(det_indices)):
        if idx == 0:
            det_indices[idx] += num_stoch
            continue
        det_indices[idx] += det_indices[idx-1]
    p = x[num_stoch:det_indices[0]]
    I = x[det_indices[0]:det_indices[1]]
    R = x[det_indices[1]:det_indices[2]]
    
    su_sl = x[det_indices[2]:]
    # scenarios*num_constraints
    su_sl = su_sl.reshape(num_scenarios*4, 2)
    
    # <det> + scenario_prob*(sum_s <stoch>) - (su_sl_cost*<su_sl>)
    det_seg = -np.dot(c,p) + np.dot(e,I) - np.dot(f,R)
    
    # Sx2 * 2xS - np.dot is matrix multiplication for 2-D arrays
    stoch_seg = np.dot(q,s.T)
    stoch_seg = np.dot(stoch_seg.diagonal(),np.ones(num_scenarios)*scenario_prob)
    
    # susl_seg = np.dot(np.dot(su_sl, c_susl), np.ones(num_scenarios*4)*scenario_prob)
    # obj = stoch_seg + det_seg - susl_seg
    
    obj = stoch_seg + det_seg
    
    return -1*obj

In [180]:
num_scenarios = 5

# surplus, slack penalties
csusl = np.array([20,20])

# sales price samples - num_scenarios*num_stoch_var
q_samples = np.random.normal(90,27,(num_scenarios, 2))

# purchase cost
c = np.array([10, 15])

# inventory coefficients
e = np.array([10, 15, 20, 25])

# operation cost
f = np.array([30, 35])

# bounds
# <det> + <sampled items> + <su/sl => 2*num_scenarios>
bounds = [(20, 40), (20, 85)]*num_scenarios # bounds unchanged for stochastic vars
bounds += [(20, 100), (30, 200), (0, 20), (0, 20), (0, 20), (0, 20), (20, 100), (30, 100)]
bounds += [(0,None),(0,None)]*num_scenarios*4 # su/sl bounds

# init vals
x0 = np.array(map(lambda x: midpoint_vals(x),bounds))

# dynamically generate equality constraints

cons = []
# last index of stochastic variables in decision array (sei)
sei = q_samples.size - 1

"""
surplus and slack variable consistent across all constraints for each scenario
determinstic_constraints = [
    {"type": "eq", "fun": lambda x: x[2] - 0.4*x[8] - x[4]},
    {"type": "eq", "fun": lambda x: x[3] - 0.6*x[8] - x[9] - x[5]},
    {"type": "eq", "fun": lambda x: 0.5*x[8] - x[6] - x[0]},
    {"type": "eq", "fun": lambda x: 0.6*x[9] - x[7] - x[1]}
]

Logic for dynamically generating constraints that use surplus and slack variables

# starting index of surplus and slack variables in decision array (ssi)
ssi = len(bounds) - (2*num_scenarios*4 + 1)
for idx in range(0,num_scenarios):
    su_idx = ssi+2*idx
    sl_idx = ssi+2*idx+1
    s1_idx = 2*idx
    s2_idx = 2*idx+1
    cons.append({"type":"eq", "fun": lambda x: x[sei+1] - 0.4*x[sei+7] - x[sei+3] + x[su_idx] - x[sl_idx]})
    cons.append({"type": "eq", "fun": lambda x: x[sei+2] - 0.6*x[sei+7] - x[sei+8] - x[sei+4] + x[su_idx] - x[sl_idx]})
    cons.append({"type": "eq", "fun": lambda x: 0.5*x[sei+7] - x[sei+5] - x[s1_idx]})
    cons.append({"type": "eq", "fun": lambda x: 0.6*x[sei+8] - x[sei+6] - x[s2_idx]})

res = optimize.minimize(tssp_obj, x0, args=(q_samples,c,e,f, csusl, num_scenarios), method="SLSQP", 
                        bounds=bounds, constraints=cons, options={"disp":True})
"""

cons.append({"type":"eq", "fun": lambda x: x[sei+1] - 0.4*x[sei+7] - x[sei+3]})
cons.append({"type": "eq", "fun": lambda x: x[sei+2] - 0.6*x[sei+7] - x[sei+8] - x[sei+4]})

# handle multiple scenarios w.r.t constraint generation
for idx in range(0,num_scenarios):
    s1_idx = 2*idx
    s2_idx = 2*idx+1    
    cons.append({"type": "eq", "fun": lambda x: 0.5*x[sei+7] - x[sei+5] - x[s1_idx]})
    cons.append({"type": "eq", "fun": lambda x: 0.6*x[sei+8] - x[sei+6] - x[s2_idx]})
    
res = optimize.minimize(tssp_obj, x0, args=(q_samples,c,e,f, csusl, num_scenarios), method="SLSQP", 
                        bounds=bounds, options={"disp":True})

x = res.x

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -12258.0130078
            Iterations: 10
            Function evaluations: 360
            Gradient evaluations: 6


### TSSP - With Financial Risk
This was not conducted due to scipy's limited capabilities with respect to handling multiobjective problems. If time permits, the multiobjective model can be written in MATLAB.

### RO SP FR

In [190]:
def rospfr_obj(x, q, c, e, f, pt, gpw):
    """
        Objective function for RO-SP-FR formulation of TSSP.
        
        Parameters
        ----------
        x : {np.array}
            design variables
        
        q : {np.array}
            stochastic sales price - shape (num_scenarios, num_det_var)
        
        c : {np.array}
            purchase costs
        
        e : {np.array}
            inventory returns
        
        f : {np.array}
            operation costs
        
        pt : {np.array}
            profit targets for each scenario - shape (num_scenarios, num_targets)
        
        gpw : {np.array}
            ROSPFR goal programming weights for each profit target - shape (num_scenarios, num_targets)
    """
    num_scenarios = len(gpw)
    scenario_prob = 1/num_scenarios
    
    # segment design variables
    num_stoch = q.size
    s = x[:num_stoch]
    s = s.reshape(num_scenarios,2)
    
    det_indices = [len(c), len(e), len(f)]
    for idx in range(0, len(det_indices)):
        if idx == 0:
            det_indices[idx] += num_stoch
            continue
        det_indices[idx] += det_indices[idx-1]
    p = x[num_stoch:det_indices[0]]
    I = x[det_indices[0]:det_indices[1]]
    R = x[det_indices[1]:]
    
    # <det> + scenario_prob*(sum_s <stoch>) - (su_sl_cost*<su_sl>)
    det_seg = -np.dot(c,p) + np.dot(e,I) - np.dot(f,R)
    
    # Sx2 * 2xS - np.dot is matrix multiplication for 2-D arrays
    stoch_seg = np.dot(q,s.T)
    stoch_seg = np.dot(stoch_seg.diagonal(), np.ones(num_scenarios)*scenario_prob)
    
    # calculated profit per scenario
    ineq_eval = np.dot(q,s.T).diagonal() + det_seg
    
    rospfr_seg = [[ineq_eval[idx] < target for target in target_arr] for idx, target_arr in enumerate(pt)]
    rospfr_seg = np.array(rospfr_seg)
    rospfr_seg = np.dot(rospfr_seg, gpw.T).diagonal()
    rospfr_seg = np.dot(rospfr_seg, np.ones(num_scenarios)*scenario_prob)
    
    obj = stoch_seg + det_seg - rospfr_seg
    
    return -1*obj

In [191]:
def rospfr_con(x, q, c, e, f, scenario, u_s, omega_i, con_type):
    """
        ROSPFR-specific constraint.
        
        Parameters
        ----------
        x : {np.array}
            design variables
        
        q : {np.array}
            sales prices - shape (num_scenarios, num_det_var)
        
        c : {np.array}
            purchase costs
        
        e : {np.array}
            inventory returns
        
        f : {np.array}
            operation costs
        
        scenario : {int}
            scenario number for indexing purposes
        
        u_s : {float}
            scenario profit upper bound
        
        omega_i : {float}
            profit target
        
        con_type : {int}
            type of constraint to return
    """
    num_scenarios = q.shape[0]
    # segment design variables
    num_stoch = q.size
    s = x[:num_stoch]
    s = s.reshape(num_scenarios,2)
    
    det_indices = [len(c), len(e), len(f)]
    for idx in range(0, len(det_indices)):
        if idx == 0:
            det_indices[idx] += num_stoch
            continue
        det_indices[idx] += det_indices[idx-1]
    p = x[num_stoch:det_indices[0]]
    I = x[det_indices[0]:det_indices[1]]
    R = x[det_indices[1]:]
    
    ineq_eval = np.dot(q[scenario],s[scenario]) - np.dot(c,p) + np.dot(e,I) - np.dot(f,R)
    
    bin_var = 1 if ineq_eval < omega_i else 0
    
    if con_type == 1:
        # q_s'*y_s - c'*x >= omega_i - u_s*z_si
        return ineq_eval + u_s*bin_var - omega_i
    else:
        # q_s'*y_s - c'*x <= omega_i + u_s(1 - z_si)
        return omega_i + u_s*(1 - bin_var) - ineq_eval

In [192]:
num_scenarios = 5
gpw_dist = (10,5) # mean and variance of penalties
profit_target = (1000, 500, 3) # mean, variance, number of targets
pt = np.random.normal(profit_target[0], profit_target[1], (num_scenarios, profit_target[2]))

# generate penalties for each of the profit targets for each of the scenarios
gpw = np.random.normal(gpw_dist[0], gpw_dist[1], (num_scenarios, profit_target[2]))

# upper bound on profits for each scenario - used in cons
u_s = np.array([1500, 2000, 3000, 5000, 1000])

# sales price samples - num_scenarios*num_stoch_var
q_samples = np.random.normal(90,27,(num_scenarios, 2))

# purchase cost
c = np.array([10, 15])

# inventory coefficients
e = np.array([10, 15, 20, 25])

# operation cost
f = np.array([30, 35])

# bounds
# <sampled stoch> + <det>
bounds = [(20, 40), (20, 85)]*num_scenarios # bounds unchanged for stochastic vars
bounds += [(20, 100), (30, 200), (0, 20), (0, 20), (0, 20), (0, 20), (20, 100), (30, 100)]

# init vals
x0 = np.array(map(lambda x: midpoint_vals(x),bounds))

cons = []
# last index of stochastic variables in decision array (sei)
sei = q_samples.size - 1
cons.append({"type":"eq", "fun": lambda x: x[sei+1] - 0.4*x[sei+7] - x[sei+3]})
cons.append({"type": "eq", "fun": lambda x: x[sei+2] - 0.6*x[sei+7] - x[sei+8] - x[sei+4]})
for idx in range(0,num_scenarios):
    s1_idx = 2*idx
    s2_idx = 2*idx+1    
    cons.append({"type": "eq", "fun": lambda x: 0.5*x[sei+7] - x[sei+5] - x[s1_idx]})
    cons.append({"type": "eq", "fun": lambda x: 0.6*x[sei+8] - x[sei+6] - x[s2_idx]})


# generate constraints for each scenario and each profit target
for s_idx in range(0, num_scenarios):
    for target in pt[s_idx]:
        cons.append({"type": "ineq", "fun": rospfr_con, "args":(q_samples,c,e,f,s_idx,u_s[s_idx],target,1)})
        cons.append({"type": "ineq", "fun": rospfr_con, "args":(q_samples,c,e,f,s_idx,u_s[s_idx],target,2)})

res = optimize.minimize(rospfr_obj, x0, args=(q_samples,c,e,f,pt,gpw), method="SLSQP", bounds=bounds, constraints=cons,
                        options={"disp":True})

x = res.x

Singular matrix C in LSQ subproblem    (Exit mode 6)
            Current function value: 184.806862553
            Iterations: 1
            Function evaluations: 20
            Gradient evaluations: 1


### Mean Markovitz Model

In [166]:
def risk_con(x, params, risk_param, profit_lb):
    """
        Risk constraint for Mean-Markovitz model.
        
        Parameters
        ----------
        x : {np.array}
            array of decision variables
            
        params : {dict}
            dictionary of parameters used for constraint generation
        
        risk_param : {float}
            custom risk parameter for variation penalty
        
        profit_lb : {float}
            lower bound for profit (t)
            
        Parameter Dictionary
        --------------------
        sales_mean : {np.array}
            mean coefficients for sales variable (selling price)
        
        sales_std : {np.array}
            standard deviation for sales variables
            
        purchase_coeff : {np.array}
            purchase cost
        
        inventory_coeff : {np.array}
            positive inventory returns
        
        operation_coeff : {np.array}
            operation costs
    """
    mu_q = params["sales_mean"]
    std_q = params["sales_std"]
    c = params["purchase_coeff"]
    e = params["inventory_coeff"]
    f = params["operation_coeff"]
    
    # segment decision variables
    s = x[:2]
    p = x[2:4]
    I = x[4:8]
    R = x[8:]
    
    const = np.dot(mu_q, s) - np.dot(c,p) + np.dot(e,I) - np.dot(f,R) - \
        risk_param*np.sqrt(np.dot(np.square(std_q),np.square(s))) - profit_lb
    
    return const

In [167]:
# profit lb
t = 100

# risk param
theta = 0.5

# mean sales price
mean_q = np.array([90, 80])

# std sales price
std_q = np.array([27, 20])

# purchase cost
c = np.array([10, 15])

# inventory coefficients
e = np.array([10, 15, 20, 25])

# operation cost
f = np.array([30, 35])

# decision variables - s3, s4, p1, p2, I1_f, I2_f, I3_f, I4_f, R1, R2

# bounds
bounds = [(20, 40), (20, 85), (20, 100), (30, 200), (0.0001, 20), (0.0001, 20), (0.0001, 20), (0.0001, 20), (20, 100), (30, 100)]

# init vals - boundary midpoints
x0 = np.array([midpoint_vals(bound) for bound in bounds])

# maximize profit
obj = lambda x: -t

params = {"sales_mean":mean_q, "sales_std": std_q, "purchase_coeff": c, "inventory_coeff": e,
          "operation_coeff": f}

# constraints
cons = (
    {"type": "ineq", "fun": risk_con, "args": (params,theta,t)},
    {"type": "eq", "fun": lambda x: x[2] - 0.4*x[8] - x[4]},
    {"type": "eq", "fun": lambda x: x[3] - 0.6*x[8] - x[9] - x[5]},
    {"type": "eq", "fun": lambda x: 0.5*x[8] - x[6] - x[0]},
    {"type": "eq", "fun": lambda x: 0.6*x[9] - x[7] - x[1]}
)

res = optimize.minimize(obj, x0, bounds=bounds, constraints=cons, options={"disp": True})

x = res.x

print "Sales Units - %s" % ", ".join(["S%s: %s" % (idx, val) for idx, val in enumerate(x[:2])])
print "Purchase Units - %s" % ", ".join(["P%s: %s" % (idx, val) for idx, val in enumerate(x[2:4])])
print "Inventory Units - %s" % ", ".join(["I%s_final: %s" % (idx, val) for idx, val in enumerate(x[4:8])])
print "Operation Levels - %s" % ", ".join(["R%s: %s" % (idx, val) for idx, val in enumerate(x[8:])])

Positive directional derivative for linesearch    (Exit mode 8)
            Current function value: -100
            Iterations: 8
            Function evaluations: 48
            Gradient evaluations: 4
Sales Units - S0: 20.0, S1: 23.1541097056
Purchase Units - P0: 34.6781443247, P1: 76.4178979509
Inventory Units - I0_final: 15.3218056753, I1_final: 18.5820520491, I2_final: 9.99999971471e-05, I3_final: 9.99999944858e-05
Operation Levels - R0: 40.0002239731, R1: 30.0
